In [1]:
import numpy as np
import matplotlib.pyplot as plt
import LavaFlow as flow

## compute effusion rate from area ##

In [2]:
## set variables ##
## environemnt
g = 8.87
theta_deg = 2
Tatm = 737

## eruption

Tlava = 1400                ## K
Tcrust = 800
rho = 2600                  ## Basalt density from Davies et al., 2006
phi0_crystal = 0.25
phi_crystal_max = 0.6       ## maximum crystal packing ratio
eta0 = 1000                 ## Pas
ys0 = 6500*phi0_crystal**2.85  

## Other variables
L = 100e3                    ## latent heat capacity in J/kg; value from Wittmann et al.2017
C =  1200                    ## specific heat capacity in J/kg/K; value from Wittman et al. 2017
k = 1                        ## thermal conductivity in W/m/K from Head and Wilson 1986, and Davies et al., 2005
kappa = k/rho/C              ## thermal diffusivity in m2/s;
Cp = 1575
Tsolid = 1253                ## from table 3 of Harris and Rowland (2001)

In [8]:
## emplacement and velocity
l = 30e3
w = 100 

area_plan = w*l
qvenus = flow.coupled_flux(Tcrust, Tatm, k, 0.95) 
qearth = flow.coupled_flux(Tcrust, 300, k, 0.95) 
E = qvenus * area_plan / (Tlava - Tsolid) / rho / Cp
heights = np.arange(5, 55, 5)
for h in heights:
    vel = E/w/h
    ## add velocity base don jeffreys equations here ##
    vel_Jeffreys = flow.velocity_Newtonian(rho, g, h, eta0, theta_deg)
    ## cooling rate
    dTdx1 = qvenus / rho / Cp / vel / 5 
    dTdx2 = qearth / rho / Cp / vel / 5 
    print(dTdx1/dTdx2)
plt.show()

0.2762580691612889
0.2762580691612889
0.2762580691612889
0.2762580691612889
0.2762580691612889
0.2762580691612889
0.276258069161289
0.2762580691612889
0.276258069161289
0.2762580691612889


In [ ]:
plt.figure(figsize = (8,8))
plt.plot()

## flowgo type modeling ##

In [8]:
effusion_rate = 10          ## m3/s

In [6]:
## compute intial conditions ##
Tsurf = Tlava
Tcore = Tlava
eta = eta0
ys = ys0
phi_crystal = phi0_crystal

## Emplacement
wb = flow.levee_width(rho, g, eta, theta_deg, ys, effusion_rate)
wc = flow.channel_width(rho, g, eta, theta_deg, ys, effusion_rate, wb)
h = flow.channel_thickness(rho, g, eta, theta_deg, ys, effusion_rate)
hb = flow.levee_thickness(rho, g, eta, theta_deg, ys, effusion_rate)
vel = flow.velocity_Bingham(rho, g, h, eta, theta_deg, ys)

## surface flux
qflynn = flow.flynn_flux(Tsurf)
qcoupled = flow.coupled_flux(Tsurf, 750, k, 0.95) 
print(vel)

0.11564621286044734


In [ ]:
## Running iterative model ##

## set model parameters
t = 0
dt  = 60
flow_length = 0.0

while (vel>=1e-4) and (Tcore>=Tsolid):
    ## Emplacement
    hb = flow.levee_thickness(rho, g, eta, theta_deg, ys, effusion_rate)
    wb = flow.levee_width(rho, g, eta, theta_deg, ys, effusion_rate)
    wc = flow.channel_width(rho, g, eta, theta_deg, ys, effusion_rate, wb)
    h = flow.channel_thickness(rho, g, eta, theta_deg, ys, effusion_rate)
    l = vel * dt
    
    ## heat loss
    qcoupled = flow.coupled_flux(Tsurf, 750, k, 0.95)  
    Told = Tcore
    
    ## new variables
    # print(qcoupled, l, wc, h, rho, vel)
    print(qcoupled * l / rho / C / vel / h)
    Tcore = Told - qcoupled * l / rho / C / vel / h
    flow_length += l

    print("viscosity = ", eta)
    print("flow length = ", flow_length, "width = ", wc, " height = ", h, " levee width = ", wb )
    eta = flow.viscosity(Tlava, Tcore, phi_crystal, phi_crystal_max, eta0)
    ys = flow.yield_strength(Tlava, Tcore, phi_crystal)
    vel = flow.velocity_Bingham(rho, g, h, eta, theta_deg, ys)
    print("velocity = ", vel)
    t+=dt
    print("Tcore=", Tcore, " velocity = ", vel,"\n")
    

In [ ]:
## comparing coupled heat loss implementations
X = np.arange(1500, 1000, -50)
flynn_2023 = flynn_flux(X) / 1e3
this_study =  coupled_flux(X, 737, k, 0.95) / 1e3
rad = radiative_flux(X, 737, 0.95) / 1e3
conv = convection_flux(X, 737, 0.95) / 1e3
plt.figure(figsize = (8,8))
plt.plot(X, flynn_2023, label = "Flynn et al. 2023")
plt.plot(X, this_study, label = "this study")
plt.plot(X, rad, label = "radiation only")
plt.plot(X, conv, label = "convection only")
plt.plot(X, conv+rad, label = "convection + radiation")
plt.xlabel("Surface temperature K")
plt.ylabel("Surface heat flux kW/m2")
plt.legend()
plt.show()

In [ ]:
## calculation of new variables
eta = viscosity(Tlava, Tcurrent, phi_crystal, phi_crystal_max, eta0)
ys = yield_strength()
vel = velocity_Bingham(rho, g, h, eta, theta_deg, yield_strength)